In [2]:
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import logging
import plotly.express as px
import pycountry
import plotly.io as pio
#pio.templates.default = "plotly_dark"

<ipython-input-2-9299ecd3f9fd>:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-2-9299ecd3f9fd>:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [3]:
df = pd.read_csv('country_data_master.csv')

In [4]:
age_df = pd.read_csv('country_data_master.csv', 
                     usecols=lambda cols: 'perc' in cols or cols == 'country' or cols == 'median_age_total')

In [5]:
age_df.head(5)

,country,median_age_total,age_0-14_perc,age_15-24_perc,age_25-54_perc,age_55-64_perc,age_65+_perc
0,Afghanistan,18.8,40.92,22.22,30.35,3.92,2.59
1,Albania,32.9,18.05,17.47,41.06,11.54,11.89
2,Algeria,28.1,29.31,15.30,42.93,6.81,5.65
3,American Samoa,25.5,30.28,18.79,36.38,8.58,5.97
4,Andorra,44.3,14.40,9.64,46.18,14.16,15.61


In [6]:
age_df = age_df.sort_values(['median_age_total'])
age_df.head(5)

,country,median_age_total,age_0-14_perc,age_15-24_perc,age_25-54_perc,age_55-64_perc,age_65+_perc
148,Niger,15.4,49.01,19.10,25.97,3.28,2.64
211,Uganda,15.8,48.05,21.10,26.30,2.57,1.98
127,Mali,15.8,48.17,18.84,26.26,3.70,3.03
5,Angola,15.9,48.12,18.25,28.03,3.26,2.34
124,Malawi,16.5,46.34,20.55,27.41,3.01,2.69


In [7]:
age_categories = ['0-14', '15-24', '25-54', '55-64', '65+']

In [8]:
input_countries = df['country'].unique()

countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_3

df['country_code'] = [countries.get(country, 'Unknown code') for country in df['country']]

fig_map = px.choropleth(df, color='median_age_total', locations='country_code', hover_name='country')


In [ ]:
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s==%(funcName)s==%(message)s')


app = dash.Dash(__name__)
server = app.server

app.layout = html.Div([
    
    html.H1('Median Age for Countries', style={'textAlign':'center', 'color':'#407A89'}),
    dcc.Graph(id='mapChart', figure=fig_map, style={'margin-left' : '-110px'}),
    dcc.Graph(id='median_age_graph', 
              config={'displayModeBar': True}),
    html.Div([
    dcc.Dropdown(id='country_dropdown',
                 multi=True,
                 value=tuple(),
                 options=[{'label': c, 'value': c}
                          for c in sorted(age_df['country'])]),        
    ], style={'width': '50%', 'margin-left': '25%','color':'#ADA3BF'}),
    dcc.Graph(id='age_graph',
             config={'displayModeBar': False}),
    

], style={'background-color': '#FFFFFF'})

@app.callback(Output('median_age_graph', 'figure'),
             [Input('country_dropdown', 'value')])
def plot_median_age(countries):
    logging.info(msg=locals())
    df = age_df[age_df['country'].isin(countries)]
    return {
        'data': [go.Scatter(x=age_df['country'],
                            y=age_df['median_age_total'],
                            mode='markers',
                            showlegend=False,
                            legendgroup='one',
                            name='',
                            hoverlabel={'font': {'size': 40}},
                            marker={'color': '#ADA3BF'})] +
                [go.Scatter(x=df[df['country']==c]['country'],
                            y=df[df['country']==c]['median_age_total'],
                            mode='markers',
                            marker={'size': 20},
                            hoverlabel={'font': {'size': 40}},
                            name=c)
                 for c in sorted(countries)],
        'layout': go.Layout(title='Median Age by Country: ' + ', '.join(countries),
                            xaxis={'title': 'Countries', 'zeroline': False, 
                                   'showticklabels': False},
                            yaxis={'title': 'Median Age', 'zeroline': False},
                            font={'family': 'Palatino','size':20,'color':'#407A89'},
                            paper_bgcolor='#FFFFFF',
                            plot_bgcolor='#FFFFFF'
),
    }

@app.callback(Output('age_graph', 'figure'),
             [Input('country_dropdown', 'value')])
def plot_countries(countries):
    df = age_df[age_df['country'].isin(countries)].sort_values('country')
    return {
        'data': [go.Bar(x=age_categories,
                        y=[0 for i in range(len(age_categories))],
                        showlegend=False,
                        width=0.1,
                        hoverinfo='none')] +
                
                [go.Bar(x=age_categories,
                        y=df.iloc[x, 2:7],
                        name=df.iloc[x, 0],
                        text=df.iloc[x, 2:7].astype(str) + '%',
                        hoverinfo='name+y',
                        textposition='inside',
                        textfont={'color': 'white'})
                 for x in range(len(df))],
        'layout': go.Layout(title='Age Distribution by Country: ' + ', '.join(countries),
                            xaxis={'title': 'Age Group Percentage', 'zeroline': False},
                            yaxis={'title': 'Percentage', 'zeroline': False},
                            font={'family': 'PT Sans Narrow','size':20,'color':'#407A89'},
                            barmode='group',
                            plot_bgcolor='#FFFFFF',
                            paper_bgcolor='#FFFFFF')
        
    }

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/



2021-11-15 14:25:12,533==run_server==Dash is running on http://127.0.0.1:8050/



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2021-11-15 14:25:12,557==_log== * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
2021-11-15 14:25:14,300==_log==127.0.0.1 - - [15/Nov/2021 14:25:14] "GET / HTTP/1.1" 200 -
2021-11-15 14:25:14,769==_log==127.0.0.1 - - [15/Nov/2021 14:25:14] "GET /_dash-dependencies HTTP/1.1" 200 -
2021-11-15 14:25:14,776==_log==127.0.0.1 - - [15/Nov/2021 14:25:14] "GET /_dash-layout HTTP/1.1" 200 -
2021-11-15 14:25:15,158==_log==127.0.0.1 - - [15/Nov/2021 14:25:15] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
2021-11-15 14:25:15,163==_log==127.0.0.1 - - [15/Nov/2021 14:25:15] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
2021-11-15 14:25:15,169==_log==127.0.0.1 - - [15/Nov/2021 14:25:15] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
2021-11-15 14:25:15,184==plot_median_age=={'countries': []}
2021-11-15 14:25:15,211==_log==127.0.0.1 - - [15/Nov/2021 14:25:15] "POST /_dash-update-component HTTP/1.1" 200 -
2021-11-15 14:2